In [2]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [3]:
# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST Dataset
train_dataset = dsets.MNIST(root='../data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='../data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

Files already downloaded


In [4]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # A sequential container, to manage the layer.
        self.layer1 = nn.Sequential(                       
            
            # Input data : (batch_size, 1, 28, 28)
            # Input channel is 1 (MNIST dataset is black and white which means channel = 1)
            # Output channel is 16
            # The inputs add 2 rows of zeros at above and bottom of the matrix 
            # And add 2 columns of zero at right and left (Cause we want 5x5)
            # Take 5x5 small matrix each time and move default stride=1 (move one pixel) to next.
            # Output data: (batch_size, 16, 28, 28)
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            
            # Input value : Num_features = 16, Input data = (batch_size, 16 , 28, 28)
            # Num_features from an expected input of (size batch_size x num_features x height x width)
            # Applies Batch Normalization
            # Output data = (batch_size, 16 , 28, 28)
            nn.BatchNorm2d(16),
            
            nn.ReLU(),
            
            # Input value : kernel_size = 2, Input data = (batch_size, 16 , 28, 28)
            # Transfer the information in 2x2 matrix into a new value
            # With default stride = kernel_size, dilation = 1  (standard convolution)
            # Input data = (batch_size, 16 , 14, 14)
            nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(
            # (batch_size, 16 , 14, 14) > (batch_size, 32, 14, 14)
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            
            #Normalize
            nn.BatchNorm2d(32),
            
            nn.ReLU(),
            
            # (batch_size, 32, 14, 14) > (batch_size, 32 ,7, 7)
            nn.MaxPool2d(2))
        
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
        
cnn = CNN()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [6]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)   # (100, 1, 28, 28)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

# Test the Model
cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Epoch [1/5], Iter [100/600] Loss: 0.2032
Epoch [1/5], Iter [200/600] Loss: 0.0918
Epoch [1/5], Iter [300/600] Loss: 0.0860
Epoch [1/5], Iter [400/600] Loss: 0.1202
Epoch [1/5], Iter [500/600] Loss: 0.1364
Epoch [1/5], Iter [600/600] Loss: 0.0269
Epoch [2/5], Iter [100/600] Loss: 0.0228
Epoch [2/5], Iter [200/600] Loss: 0.0448
Epoch [2/5], Iter [300/600] Loss: 0.0263
Epoch [2/5], Iter [400/600] Loss: 0.0384
Epoch [2/5], Iter [500/600] Loss: 0.1155
Epoch [2/5], Iter [600/600] Loss: 0.0702
Epoch [3/5], Iter [100/600] Loss: 0.0026
Epoch [3/5], Iter [200/600] Loss: 0.0523
Epoch [3/5], Iter [300/600] Loss: 0.0208
Epoch [3/5], Iter [400/600] Loss: 0.0085
Epoch [3/5], Iter [500/600] Loss: 0.0123
Epoch [3/5], Iter [600/600] Loss: 0.0064
Epoch [4/5], Iter [100/600] Loss: 0.0246
Epoch [4/5], Iter [200/600] Loss: 0.0242
Epoch [4/5], Iter [300/600] Loss: 0.0238
Epoch [4/5], Iter [400/600] Loss: 0.0112
Epoch [4/5], Iter [500/600] Loss: 0.0157
Epoch [4/5], Iter [600/600] Loss: 0.0141
Epoch [5/5], Ite

In [ ]:
# Save the Trained Model
torch.save(cnn.state_dict(), 'cnn.pkl')